In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import nltk

nltk.download('stopwords')
import numpy as np
from nltk.stem import LancasterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import os

lancStemmer = LancasterStemmer()  # stemming algorithm Lancaster


# lemmatizer = WordNetLemmatizer() #lemmatizer algorithm

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sergiogonzalez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palabras = [palabra.lower() for palabra in re.findall(r'\w+', text.lower())]
    text_lista = []
    for palabra in palabras:
        if palabra not in stopwords:
            text_lista.append(palabra)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

In [3]:

def get_stemmer(text):
    palabras = remove_stopwords(text)
    palabras = palabras.split()
    text_lista = []
    for palabra in palabras:
        nueva = lancStemmer.stem(palabra)
        text_lista.append(nueva)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

In [4]:

def get_grams(text, n):
    text = get_stemmer(text)  # pre-procesa el parrafo
    text = text.split()  # separa los caracteres pre-procesados del parrafo en listas
    grams = ngrams(text, n)  # genera los ngrams
    result = []
    for ng in grams:
        result.append(' '.join(ng))  # agrega los ngrams en una lista llamada result
    return result

In [5]:

def pre_procesados (folder_path, n):
  preprocess_texts = []
  for fileid in os.listdir(folder_path):
    if fileid.endswith(".txt"):
      filepath = os.path.join(folder_path, fileid)
      with open(filepath, 'r', encoding='latin1', errors='ignore') as file:
        text = file.read()
        grams = get_grams(text, n)
        preprocess_texts.append((fileid, grams))
  return preprocess_texts

In [6]:
def matriz_parrafos(grams1, grams2):
    grams_palabras = set(grams1 + grams2)  # set de palabras de ambos ngrams
    grams_juntos = [grams1, grams2]  # lista con ambas listas de los ngrams de cada parrafo
    matriz = []
    for grama in grams_juntos:
        vector = []
        for palabra in grams_palabras:
            vector.append(
                1 if palabra in grama else 0)  # compara las palabras de los grams a la palabra y agrega 1 o 0 al vector del parrafo
        matriz.append(vector)
    return matriz

In [56]:
# Obtener n-gramas preprocesados
folder_path = "/Users/sergiogonzalez/Documents/GitHub/DetectorPlagio/textos_plagiados" # Ruta de la carpeta con los textos plagiados
preprocess_plagiados = pre_procesados(folder_path, 3)

folder_path_og = "/Users/sergiogonzalez/Documents/GitHub/DetectorPlagio/docs_originales" # Ruta de la carpeta con los textos originales
preprocess_originales = pre_procesados(folder_path_og, 3)

for id_plagiado, (name_plagiado, grams_plagiado) in enumerate(preprocess_plagiados, 1):
    print(f'\nDocumento analizado: {name_plagiado}')
    for id_original, (name_original, grams_original) in enumerate(preprocess_originales, 1):
        similitud = cosine_similarity(matriz_parrafos(grams_plagiado, grams_original))
        print(f"Similitud de Coseno entre {name_plagiado} y {name_original}: {similitud[0][1]}")


Documento analizado: FID-06.txt
Similitud de Coseno entre FID-06.txt y org-107.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-098.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-067.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-073.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-072.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-066.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-099.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-106.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-104.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-110.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-058.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-070.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-064.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-065.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-071.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-059.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-105.txt: 0.0
Similitud de Co

### LAB CADENAS DE OCURRENCIA

In [57]:
resultados = []
for id_plagiado, (name_plagiado, grams_plagiado) in enumerate(preprocess_plagiados, 1):
    for id_original, (name_original, grams_original) in enumerate(preprocess_originales, 1):
        similitud = cosine_similarity(matriz_parrafos(grams_plagiado, grams_original))
        if similitud[0][1] != 0.0 and similitud[0][1] >= 0.2:
            resultados.append([name_plagiado, name_original, similitud[0][1]])
            

In [70]:
resultados

[['FID-05.txt', 'org-085.txt', 0.9999999999999983],
 ['FID-03.txt', 'org-016.txt', 0.9173191625339879],
 ['FID-04.txt', 'org-045.txt', 0.9055118110236218],
 ['FID-08.txt', 'org-079.txt', 0.8382349847675461],
 ['FID-09.txt', 'org-109.txt', 0.808248929265168],
 ['FID-02.txt', 'org-104.txt', 0.6631210306331439],
 ['FID-01.txt', 'org-076.txt', 0.6468162120117367],
 ['FID-07.txt', 'org-041.txt', 0.5918092276655763],
 ['FID-10.txt', 'org-007.txt', 0.5786876586795143],
 ['FID-06.txt', 'org-043.txt', 0.5574519604163601]]

### Resultados Esperados
1-. una lista ordenada,descendentemente por similitud, donde cada entrada incluye, para cada documento asociado.

2-. puntaje de similitud.

3-. cadena de coincidencia.

4-. longitud y el apuntador que nos lleva a su ocurrencia.

In [73]:
resultados.sort(key=lambda x: x[2], reverse=True)
for resultado in resultados:
    print("resultado: ", resultado[0], resultado[1])
    for original in preprocess_originales:
        if resultado[1] in original:
            print(f"Encontrado {resultado[1]} en {original}")
    for plagiados in preprocess_plagiados:
        if plagiados[0] == resultado[0]:
            print(f"Encontrado {resultado[0]} en {plagiados}")
            print("----------------------------\n")

resultado:  FID-05.txt org-085.txt
Encontrado org-085.txt en ('org-085.txt', ['internet thing iot', 'thing iot bas', 'iot bas remot', 'bas remot healthc', 'remot healthc apply', 'healthc apply provid', 'apply provid fast', 'provid fast prev', 'fast prev med', 'prev med serv', 'med serv paty', 'serv paty risk', 'paty risk howev', 'risk howev predict', 'howev predict heart', 'predict heart diseas', 'heart diseas complex', 'diseas complex task', 'complex task diagnos', 'task diagnos result', 'diagnos result rar', 'result rar acc', 'rar acc address', 'acc address issu', 'address issu novel', 'issu novel recommend', 'novel recommend system', 'recommend system cardiovascul', 'system cardiovascul diseas', 'cardiovascul diseas predict', 'diseas predict us', 'predict us iot', 'us iot network', 'iot network deep', 'network deep cardio', 'deep cardio propos', 'cardio propos provid', 'propos provid pri', 'provid pri diagnos', 'pri diagnos tre', 'diagnos tre diet', 'tre diet recommend', 'diet recom

In [121]:
resultados.sort(key=lambda x: x[2], reverse=True)
lista_titulos = []
for resultado in resultados:
    for plagiados in preprocess_plagiados:
        if plagiados[0] == resultado[0]:
            lista_titulos.append([plagiados[0], resultado[1]])
  

In [119]:
lista_titulos

[['FID-05.txt', 'org-085.txt'],
 ['FID-03.txt', 'org-016.txt'],
 ['FID-04.txt', 'org-045.txt'],
 ['FID-08.txt', 'org-079.txt'],
 ['FID-09.txt', 'org-109.txt'],
 ['FID-02.txt', 'org-104.txt'],
 ['FID-01.txt', 'org-076.txt'],
 ['FID-07.txt', 'org-041.txt'],
 ['FID-10.txt', 'org-007.txt'],
 ['FID-06.txt', 'org-043.txt']]